# Global Supermaket Data Recsys

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/%5BDArt-B%205%EA%B8%B0%5D%20Global_Supermarket.csv')
data.head()

,customer_id,customer_name,customer_segment,order_id,order_city,oreder_region,order_date,order_year,order_weeknum,quantity,...,discount,category,sub_category,market_country,market_area,market_city,ship_date,ship_mode,shipping_cost,row_id
0,LS-172304,Lycoris Saunders,Consumer,CA-2019-130813,Los Angeles,West,2019-01-07,2019,2,3,...,0.0,Office Supplies,Paper,United States,US,California,2019-01-09,Second Class,4.37,36624
1,MV-174854,Mark Van Huff,Consumer,CA-2019-148614,Los Angeles,West,2019-01-21,2019,4,2,...,0.0,Office Supplies,Paper,United States,US,California,2019-01-26,Standard Class,0.94,37033
2,CS-121304,Chad Sievert,Consumer,CA-2019-118962,Los Angeles,West,2019-08-05,2019,32,3,...,0.0,Office Supplies,Paper,United States,US,California,2019-08-09,Standard Class,1.81,31468
3,CS-121304,Chad Sievert,Consumer,CA-2019-118962,Los Angeles,West,2019-08-05,2019,32,2,...,0.0,Office Supplies,Paper,United States,US,California,2019-08-09,Standard Class,4.59,31469
4,AP-109154,Arthur Prichep,Consumer,CA-2019-146969,Los Angeles,West,2019-09-29,2019,40,1,...,0.0,Office Supplies,Paper,United States,US,California,2019-10-03,Standard Class,1.32,32440


## User-Item Matrix

In [2]:
from sklearn.preprocessing import LabelEncoder

# Create a user-item matrix
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

data['customer_id_encoded'] = user_encoder.fit_transform(data['customer_id'])
data['product_id_encoded'] = item_encoder.fit_transform(data['product_id'])

user_item_matrix = data.groupby(['customer_id_encoded', 'product_id_encoded'])['quantity'].sum().unstack()
item_user_matrix = data.groupby(['product_id_encoded','customer_id_encoded'])['quantity'].sum().unstack()

user_item_matrix

product_id_encoded,0,1,2,3,4,5,6,7,8,9,...,10282,10283,10284,10285,10286,10287,10288,10289,10290,10291
customer_id_encoded,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Item-Based CF
- Find Similarity Score btw Items
- Select K items for reference.(k ~= 20)
- For Items, Evalue final prediction score 
- making Top-N recommendation for target user.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
item_user_matrix_na = item_user_matrix.fillna(0)
item_cos_score = cosine_similarity(item_user_matrix_na)

pd.DataFrame(item_cos_score)

,0,1,2,3,4,5,6,7,8,9,...,10282,10283,10284,10285,10286,10287,10288,10289,10290,10291
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
prediction = csr_matrix(item_user_matrix_na)

import numpy as np
def predict_rating(uim,sim_matrix,item_index,user_index):
    uim = csr_matrix(uim)
    sim_matrix = csr_matrix(sim_matrix)
    item_score = uim.iloc[user_index,:].values
    sim_score = sim_matrix[item_index]

    rated_ind = np.where(item_score>0)
    numerator = np.sum(sim_score[rated_ind] * item_score[rated_ind])
    denominator = np.sum(sim_score[rated_ind])

    if denominator == 0:
        return 0
    return denominator / numerator

for user_index in range(len(item_user_matrix.columns)):
    for item_index in range(len(item_user_matrix.index)):
        if item_user_matrix.iloc[item_index,user_index] == 0:
            prediction.iloc[user_index,item_index] = predict_rating(item_user_matrix_na,item_cos_score,item_index,user_index)

AttributeError: 'csr_matrix' object has no attribute 'head'

In [ ]:
#Item-User Matrix
pd.DataFrame(prediction.todense())

,0,1,2,3,4,5,6,7,8,9,...,4863,4864,4865,4866,4867,4868,4869,4870,4871,4872
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## User-Based CF

In [4]:
user_item_matrix_na = user_item_matrix.fillna(0)

- User_pearson Score Represents the Correlation(Pearson Similarity) btw each Users.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
user_item_matrix_sparse = csr_matrix(user_item_matrix_na)
user_cosine_score = cosine_similarity(user_item_matrix_sparse)
pd.DataFrame(user_cosine_score)

,0,1,2,3,4,5,6,7,8,9,...,4863,4864,4865,4866,4867,4868,4869,4870,4871,4872
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.038602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.164384,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


- user간 similarity score
- user랑 비슷한 유저 선택(k),해당 유저에 맞는 item 선택(N)
- k*N matrix를 이용해서 해당 user에 대한 item의 rating 계산


In [ ]:
def get_similar_users(user_id,user_cosine_score,k,N):
    similar_users = user_cosine_score[user_id].sort_values(ascending=False)[1:k+1]
    
